In [1]:
%load_ext dotenv
%dotenv ../keys.env

from requests import Request,Session,Response
import time
import datetime
import json
import logging
import hmac
import hashlib
import os
from dotenv import load_dotenv
import urllib
from typing import Optional, Dict, Any, List

In [2]:
os.getenv('FTX_MAIN_KEY')
os.getenv('FTX_MAIN_SECRET')

'K-FB2PF_vSdyA-fQu41ElR6PuVpq-VScaBxy1bF_'

In [2]:
from colorama import Fore, Back, Style, init
logging.basicConfig(level=logging.INFO, format=(
    Fore.BLUE + '[+] ' + Style.RESET_ALL + '%(message)s '))
logger = logging.getLogger("Logger")

In [16]:
class FtxClient:
    _ENDPOINT = 'https://ftx.com/api/'
    markets = {
        "ATOM":"ATOM-PERP",
        "XTZ":"XTZ-PERP",
        "BTC":"BTC-PERP",
        "ETH":"ETH-PERP"
        
    }
    def __init__(self,subaccount_name = None) -> None:
        self._session = Session()
        self._api_key = os.getenv('FTX_API_KEY')
        self._api_secret = os.getenv('FTX_API_SECRET')
        self._subaccount_name = subaccount_name
    
    def _get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        return self._request('GET', path, params=params)
    
    def _request(self,method:str,path:str, **kwargs) -> Any:
        request = Request(method,self._ENDPOINT + path, **kwargs)
        # Apply hash to keys
        self._sign_request(request)
        # Send the request, similar to req.get() or req.post()
        response = self._session.send(request.prepare())
        # Clean up response
        result = self._process_response(response)
        for item in result:
            name = item['name']
            logger.info(f'{name}')
    def _sign_request(self,request:Request) -> None:
        ts = int(time.time() * 1000)
        prepared = request.prepare()
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        
        if prepared.body:
            signature_payload += prepared.body
        logger.info(f'{signature_payload}')
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()
        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)
        if self._subaccount_name:
            request.headers['FTX-SUBACCOUNT'] = urllib.parse.quote(self._subaccount_name)
    def _process_response(self, response: Response) -> Any:
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    def get_account_info(self) -> dict:
        return self._get(f'account')
    def list_markets(self) -> List[dict]:
        return self._get('markets')
    def place_order(self,market):
        pass

SyntaxError: invalid syntax (<ipython-input-16-a030a23f2184>, line 9)

In [13]:
ftx = FtxClient(subaccount_name = 'hunter-api')

In [11]:
ftx.get_account_info()

[+] b'1595104065613GET/api/account' 
[+] {'backstopProvider': False, 'chargeInterestOnNegativeUsd': False, 'collateral': 96.39843783843, 'freeCollateral': 93.92668302206, 'initialMarginRequirement': 0.1, 'leverage': 10.0, 'liquidating': False, 'maintenanceMarginRequirement': 0.03, 'makerFee': 0.00019, 'marginFraction': None, 'openMarginFraction': None, 'positionLimit': None, 'positionLimitUsed': None, 'positions': [], 'spotLendingEnabled': False, 'spotMarginEnabled': False, 'takerFee': 0.000665, 'totalAccountValue': 96.39843783843, 'totalPositionSize': 0.0, 'useFttCollateral': True, 'username': 'deerhunter001@protonmail.com/hunter-api'} 


In [14]:
ftx.list_markets()

[+] b'1595105518934GET/api/markets' 
[+] ADA-PERP 
[+] ADA-0925 
[+] ALGO-PERP 
[+] ALGO-0925 
[+] ALT-PERP 
[+] ALT-0925 
[+] ATOM-PERP 
[+] ATOM-0925 
[+] BAL-PERP 
[+] BAL-0925 
[+] BAL/USD 
[+] BAL/USDT 
[+] BCH-PERP 
[+] BCH-0925 
[+] BCH/USD 
[+] BCH/USDT 
[+] BNB-PERP 
[+] BNB-0925 
[+] BNB/USD 
[+] BNB/USDT 
[+] BRL/USD 
[+] BRL/USDT 
[+] BRZ-PERP 
[+] BRZ-0925 
[+] BSV-PERP 
[+] BSV-0925 
[+] BTC-PERP 
[+] BTC-MOVE-0718 
[+] BTC-MOVE-0719 
[+] BTC-MOVE-WK-0724 
[+] BTC-MOVE-WK-0731 
[+] BTC-MOVE-WK-0807 
[+] BTC-MOVE-WK-0814 
[+] BTC-0925 
[+] BTC-MOVE-2020Q3 
[+] BTC-1225 
[+] BTC-MOVE-2020Q4 
[+] BTC-MOVE-2021Q1 
[+] BTC/BRL 
[+] BTC/USD 
[+] BTC/USDT 
[+] BTC-HASH-2020Q3 
[+] BTC-HASH-2020Q4 
[+] BTC-HASH-2021Q1 
[+] BTMX-PERP 
[+] BTMX-0925 
[+] BTMX/USD 
[+] COMP-PERP 
[+] COMP-0925 
[+] COMP/USD 
[+] COMP/USDT 
[+] CUSDT-PERP 
[+] CUSDT-0925 
[+] CUSDT/USD 
[+] CUSDT/USDT 
[+] DEFI-PERP 
[+] DEFI-0925 
[+] DMG-PERP 
[+] DMG-0925 
[+] DMG/USD 
[+] DMG/USDT 
[+] DOGE-PERP 

[+] TRXBEAR/USD 
[+] TRXBULL/USD 
[+] TRXHALF/USD 
[+] TRXHEDGE/USD 
[+] TRYBBEAR/USD 
[+] TRYBBULL/USD 
[+] TRYBHALF/USD 
[+] TRYBHEDGE/USD 
[+] USDTBEAR/USD 
[+] USDTBULL/USD 
[+] USDTHALF/USD 
[+] USDTHEDGE/USD 
[+] VETBEAR/USD 
[+] VETBEAR/USDT 
[+] VETBULL/USD 
[+] VETBULL/USDT 
[+] VETHEDGE/USD 
[+] XAUTBEAR/USD 
[+] XAUTBULL/USD 
[+] XAUTHALF/USD 
[+] XAUTHEDGE/USD 
[+] XRPBEAR/USD 
[+] XRPBEAR/USDT 
[+] XRPBULL/USD 
[+] XRPBULL/USDT 
[+] XRPHALF/USD 
[+] XRPHEDGE/USD 
[+] XTZBEAR/USD 
[+] XTZBEAR/USDT 
[+] XTZBULL/USD 
[+] XTZBULL/USDT 
[+] XTZHALF/USD 
[+] XTZHEDGE/USD 
